In [1]:
# (ONLY run this cell if running notebook from BROWSER. instead of IDE):
import sys, os 
proj_root = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)
from utils import api_callr as api
from utils import cif_parsr as cp

import glob
import json
from Bio.PDB.MMCIF2Dict import MMCIF2Dict

In [2]:
# # (ONLY run this cell if running notebook from IDE. instead of BROWSER):
# import os
# import glob
# import json
# from Bio.PDB.MMCIF2Dict import MMCIF2Dict
# from utils import api_callr as api
# from utils import cif_parsr as cp
# # %load_ext autoreload
# # %autoreload 2

In [3]:
# # GET RAW mmCIF FILES FOR ALL SOLUTION NMR DATA
# %load_ext autoreload
# %autoreload 2

# relpath = '../data/NMR/raw_cifs/'

# meric = 'homomeric/'
# sol_nmr_homo_686_pdbids = api.call_rcsb_for_pdbids_of_solution_nmr_homomeric(number=686)
# for homo_pdbid in sol_nmr_homo_686_pdbids:
#     response = api.call_rcsb_for_cif(homo_pdbid)
#     with open(f'{relpath}{meric}{homo_pdbid}.cif', 'w') as cif_file:
#         cif_file.write(response.text)

# meric = 'heteromeric/'
# sol_nmr_hetero_1038_pdbids = api.call_rcsb_for_pdbids_of_solution_nmr_heteromeric(number=1038)
# for hetero_pdbid in sol_nmr_hetero_1038_pdbids:
#     response = api.call_rcsb_for_cif(hetero_pdbid)
#     with open(f'{relpath}{meric}/{hetero_pdbid}.cif', 'w') as cif_file:
#         cif_file.write(response.text)

In [ ]:
# CONVERT RAW mmCIFs TO DICTS. PARSE & TOKENISE THESE:
# %load_ext autoreload
# %autoreload 2
meric = 'homomeric'

relpath_homo_cifs = glob.glob(os.path.join(f'../data/NMR/raw_cifs/{meric}', f'*.cif'))

for relpath_cif in relpath_homo_cifs:
    cif_dict = MMCIF2Dict(relpath_cif)
    pdbid = os.path.basename(relpath_cif).removesuffix('.cif')
    cif_pdfs_per_chain = cp.parse_cif(pdb_id=pdbid, mmcif_dict=cif_dict)  # AS MSC PROJECT BUT ALPHA-CARBONS ONLY.

    for pdf_chain in cif_pdfs_per_chain:
        if pdbid == '1BBX':
            print(f'{pdbid}={pdf_chain}')

        with open('../data/enumeration/residues.json', 'r') as json_f:
            residues_enumerated = json.load(json_f)
        pdf_chain = pdf_chain.copy()
        pdf_chain.loc[:, 'aa_label_num'] = pdf_chain['S_mon_id'].map(residues_enumerated).astype('Int64')

        if not pdf_chain.empty:
            chain = pdf_chain['S_asym_id'].iloc[0]
        else:
            print(f'pdf_chain is empty? --> {pdf_chain}')
            continue

        pdf_chain = pdf_chain[['A_pdbx_PDB_model_num', 'S_seq_id', 'S_mon_id', 'aa_label_num', 'A_id',
                               'A_Cartn_x', 'A_Cartn_y', 'A_Cartn_z']]
        
        expected_num_of_cols = 8
        assert len(pdf_chain.columns) == expected_num_of_cols, \
        (f'Dataframe should have {expected_num_of_cols} columns. '
         f'But this has {len(pdf_chain.columns)}')

        pdf_chain.to_csv(path_or_buf=f'../data/NMR/tokenised_cifs/{meric}/{pdbid}_{chain}.ssv', sep=' ', index=False)

Parse 1A03
pdf.shape after removing everything except alpha-carbons=(1800, 15)
pdf.shape after removing everything except alpha-carbons=(1800, 15)
Parse 1A1U
pdf.shape after removing everything except alpha-carbons=(29, 15)
pdf.shape after removing everything except alpha-carbons=(29, 15)
Parse 1AFO
pdf.shape after removing everything except alpha-carbons=(800, 15)
pdf.shape after removing everything except alpha-carbons=(800, 15)
Parse 1AJY
pdf.shape after removing everything except alpha-carbons=(923, 15)
pdf.shape after removing everything except alpha-carbons=(923, 15)
Parse 1AQ5
pdf.shape after removing everything except alpha-carbons=(940, 15)
pdf.shape after removing everything except alpha-carbons=(940, 15)
pdf.shape after removing everything except alpha-carbons=(940, 15)
Parse 1ARQ
pdf.shape after removing everything except alpha-carbons=(848, 15)
pdf.shape after removing everything except alpha-carbons=(848, 15)
Parse 1ARR
pdf.shape after removing everything except alpha-car

In [ ]:
meric = 'heteromeric'

relpath_homo_cifs = glob.glob(os.path.join(f'../data/NMR/raw_cifs/{meric}', f'*.cif'))
for relpath_cif in relpath_homo_cifs:
    cif_dict = MMCIF2Dict(relpath_cif)
    pdbid = os.path.basename(relpath_cif).removesuffix('.cif')
    cif_pdfs_per_chain = cp.parse_cif(pdb_id=pdbid, mmcif_dict=cif_dict)  # AS MSC PROJ BUT ALPHA-CARBONS ONLY.

    for pdf_chain in cif_pdfs_per_chain:
        # ENUMERATE RESIDUES:
        with open('../data/enumeration/residues.json', 'r') as json_f:
            residues_enumerated = json.load(json_f)
        pdf_chain = pdf_chain.copy()
        pdf_chain.loc[:, 'aa_label_num'] = pdf_chain['S_mon_id'].map(residues_enumerated).astype('Int64')

        # GET CHAIN AND MODEL NUM FOR FILENAME (BEFORE DROPPING COLUMN):
        chain = pdf_chain['S_asym_id'].iloc[0]

        # REORDER COLUMNS:
        pdf_chain = pdf_chain[['A_pdbx_PDB_model_num', 'S_seq_id', 'S_mon_id',  'aa_label_num', 'A_id',
                               'A_Cartn_x', 'A_Cartn_y', 'A_Cartn_z']]
        expected_num_of_cols = 8
        assert len(pdf_chain.columns) == expected_num_of_cols, \
        (f'Dataframe should have {expected_num_of_cols} columns. '
         f'But this has {len(pdf_chain.columns)}')

        pdf_chain.to_csv(path_or_buf=f'../data/NMR/tokenised_cifs/{meric}/{pdbid}_{chain}.ssv',
                         sep=' ', index=False)